<a href="https://colab.research.google.com/github/Nick088Official/versatile_audio_super_resolution/blob/main/Audio_SR_NO_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio SR NO UI

Versatile Audio Super Resolution (any -> 48kHz)

*NOTE:* Be sure to be using a GPU, it will crash on CPU because of the free RAM being too little

In [ ]:
#@title Install
import os
from IPython.display import clear_output
!pip install audiosr
os.mkdir("Audio_SR")
%cd Audio_SR
os.mkdir("Inputs")
os.mkdir("Outputs")
Drive_Is_Mounted = False
parent_dir = '/content/Audio_SR'
inputs_path = "/content/Audio_SR/Inputs/"
clear_output()
print("Installed!")

In [ ]:
#@title (OPTIONAL) Mount Google Drive

#@markdown NOTE: If you run this, the rest of the colab will use your Google Drive inputs and outputs folder instead of the Google Colab ones.

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the parent directory path
parent_dir = '/content/drive/MyDrive/Audio_SR'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_dir):
    os.mkdir(parent_dir)

# Create the 'Inputs' and 'Outputs' subdirectories
inputs_dir = os.path.join(parent_dir, 'Inputs')
outputs_dir = os.path.join(parent_dir, 'Outputs')

if not os.path.exists(inputs_dir):
  os.mkdir(inputs_dir)

if not os.path.exists(outputs_dir):
  os.mkdir(outputs_dir)

inputs_path = "/content/drive/MyDrive/Audio_SR/Inputs/"

Drive_Is_Mounted = True

In [ ]:
#@title Upload Input Audio

#@markdown NOTE: If you Runned the Mount Google Drive Cell, it will upload the inputs into Google Drive. Also this won't work for NON Google Chrome Users, check the 2nd step for non chrome users in the guide if you are one of those.

from google.colab import files
import os
import shutil

# upload
uploaded = files.upload()

if Drive_Is_Mounted:

  target_directory = '/content/drive/MyDrive/Audio_SR/Inputs'

  # Get a list of all files uploaded by the user
  user_uploaded_files = list(uploaded.keys())

  # Move each user-uploaded file to the target directory
  for file_name in user_uploaded_files:
      source_path = os.path.join(f'/content/Audio_SR', file_name)
      target_path = os.path.join(target_directory, file_name)
      shutil.move(source_path, target_path)
      print(f"Moved '{file_name}' to the Google Drive Audio_SR/Inputs folder.")
else:

  target_directory = '/content/Audio_SR/Inputs'

  # Get a list of all files uploaded by the user
  user_uploaded_files = list(uploaded.keys())

  # Move each user-uploaded file to the target directory
  for file_name in user_uploaded_files:
      source_path = os.path.join(f'/content/Audio_SR', file_name)
      target_path = os.path.join(target_directory, file_name)
      shutil.move(source_path, target_path)
      print(f"Moved '{file_name}' to the Audio_SR/Inputs folder.")

In [ ]:
#@title Run Inference

input_mode = "single file" #@param ["single file", "file list"]

#@markdown **FOR SINGLE FILE MODE:** Input audio filename (with extension) for audio super resolution
input_audio_file = "low quality.wav" #@param {type:"string"}

#@markdown **FOR FILE LIST MODE:** A filename (with extension) that contains all audio files that need to perform audio super resolution
input_file_list = "batch.lst" #@param {type:"string"}

#@markdown The path to save model output will be /content/Audio_SR/Outputs, if you Mounted Google Drive it will be in the Audio_SR Google Drive Output Folder instead.

#@markdown The checkpoint you gonna use
model_name = "speech" #@param ["basic", "speech"]

#@markdown The sampling step for DDIM
ddim_steps = 50 #@param {type:"slider", min:1, max:100, step:1}

#@markdown Guidance scale (Large => better quality and relavancy to text; Small => better diversity)
guidance_scale = 3.5 #@param {type:"slider", min:0.0, max:10.0, step:0.1}

#@markdown Changing this value (any integer number) will lead to a different generation result, put 0 for random seed.
seed = 42 #@param {type:"slider", min:0, max:4294967295, step:1}

#@markdown Suffix of the Output file
suffix = "_AudioSR_Processed_48K" #@param {type:"string"}

input_audio_file = f"{inputs_path}{input_audio_file}"
input_file_list = f"{inputs_path}{input_file_list}"

if Drive_Is_Mounted:
  save_path = '/content/drive/MyDrive/Audio_SR/Outputs/'
else:
  save_path = '/content/Audio_SR/Outputs/'

# set random seed when seed input value is 0
if seed == 0:
  import random
  seed = random.randint(1, 2**32-1)

if input_mode == "single file":
  command = f"-i '{input_audio_file}' -s {save_path} --model_name {model_name} --ddim_steps {ddim_steps} -gs {guidance_scale} --seed {seed} --suffix {suffix}"
elif input_mode == "file list":
  command = f"-il '{input_file_list}' -s {save_path} --model_name {model_name} --ddim_steps {ddim_steps} -gs {guidance_scale} --seed {seed} --suffix {suffix}"

!audiosr $command

In [ ]:
#@title Display Result
from IPython.display import Audio, display, clear_output
import os
import glob

# Store the current directory
original_dir = %pwd

# join save path
%cd $save_path

# Remove the extension and input path from the original filename
input_audio_file = input_audio_file.replace(f"{inputs_path}", "")
input_filename_without_extension = os.path.splitext(input_audio_file)[0]

# Find all timestamped folders
timestamped_folders = glob.glob("*_*_*_*_*_*")

# Sort folders by modification time (newest first)
timestamped_folders.sort(key=os.path.getmtime, reverse=True)

# Get the most recent folder
most_recent_folder = timestamped_folders[0] if timestamped_folders else None

if most_recent_folder:
    # Construct the full path to the output file
    output_file_path = os.path.join(most_recent_folder, input_filename_without_extension + suffix + os.path.splitext(input_audio_file)[1])

    # Load the audio data
    import librosa
    audio_data, _ = librosa.load(output_file_path, sr=48000)  # Load audio at 48kHz

    # Change back to the original directory
    %cd $original_dir

    # Clear the output
    clear_output()

    # Display the audio
    display(Audio(audio_data, rate=48000, autoplay=True)) # Pass the audio data instead of the filename
else:
    print("No output folders found in the output directory.")

In [ ]:
#@title Dowload Output Result

#@markdown NOTE: If you Runned the Mount Google Drive Cell, it will download automatically the output from Google Drive Audio_SR/Outputs folder.

from google.colab import files

files.download(f"{save_path}{output_file_path}")

In [ ]:
#@title Delete Inputs & Ouputs

#@markdown NOTE: You may wanna do this just to free up space, especially if you mounted Google Drive

import os
import shutil

# Save the current working directory
original_dir = os.getcwd()

# Change to the parent directory
%cd $parent_dir

# Use shutil.rmtree to remove directories and their contents
shutil.rmtree("Inputs", ignore_errors=True) # ignore errors if the directory doesn't exist
shutil.rmtree("Outputs", ignore_errors=True)

os.mkdir("Inputs")
os.mkdir("Outputs")

print("Succesfully cleared Inputs & Outputs folders!")

# Return to the original directory
%cd $original_dir